## BERT Image Attributes and Product Dicreiption Vector Generator

### 1. Loading Pre-Trained BERT

- Pre-train model: bert-large-uncased
-For 24-layer, 1024-hidden, 16-heads, 340M parameters. Trained on lower-cased English text.

In [1]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 5.0MB/s 


In [3]:
from fastai.vision import *

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 1247007.65B/s]


### Load Atrributes and product Description

In [0]:
path = Path('/content/drive/My Drive/Levis')
path_modal = path /'multimodal-search'
levis_attri_desc = pd.read_csv(path_modal/'levis_img_attr_label_desc.csv')

In [5]:
levis_attri_desc.head()

,description,label,file_path,all_attributes
0,exact reproduction of 1961 551z customized...,551 z customized men's jeans,downloaded_images/748790000___748790000-back-p...,"straight,zip,slim fit,medium wash,freewheelin ..."
1,exact reproduction of 1961 551z customized...,551 z customized men's jeans,downloaded_images/748790000___748790000-side-p...,"straight,zip,slim fit,medium wash,freewheelin ..."
2,exact reproduction of 1961 551z customized...,551 z customized men's jeans,downloaded_images/748790000___748790000-alt6-p...,"straight,zip,slim fit,medium wash,freewheelin ..."
3,exact reproduction of 1961 551z customized...,551 z customized men's jeans,downloaded_images/748790000___748790000-front-...,"straight,zip,slim fit,medium wash,freewheelin ..."
4,perfect fit for athletic builds relaxed...,541 athletic taper fit camo patch men's jeans,downloaded_images/181810426___181810426-front-...,"athletic,taper,relaxed,541 ,zip,sits at waist,..."


### Input 

In [6]:
attr_text = levis_attri_desc['all_attributes'][0]
attr_text

"straight,zip,slim fit,medium wash,freewheelin suze , medium wash,men's regular,non stretch,100% cotton,non,stretch denim,imported"

In [7]:
attr_text_clean = attr_text.replace(',',' ')
attr_text_clean

"straight zip slim fit medium wash freewheelin suze   medium wash men's regular non stretch 100% cotton non stretch denim imported"

In [0]:
marked_text = "[CLS] " + attr_text_clean + " [SEP]"

In [11]:
# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
straight      3,442
zip          14,101
slim         11,754
fit           4,906
medium        5,396
wash          9,378
free          2,489
##wheel      22,920
##in          2,378
su           10,514
##ze          4,371
medium        5,396
wash          9,378
men           2,273
'             1,005
s             1,055
regular       3,180
non           2,512
stretch       7,683
100           2,531
%             1,003
cotton        6,557
non           2,512
stretch       7,683
denim        26,762
imported     10,964
[SEP]           102


In [12]:
# Mark each of tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Extract Embeddings

In [13]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
# model = BertModel.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-large-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

100%|██████████| 1248501532/1248501532 [00:31<00:00, 39480260.33B/s]


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (

In [0]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [15]:
print ("Number of layers:", len(encoded_layers))
layer_i = 0

print ("Number of batches:", len(encoded_layers[layer_i]))
batch_i = 0

print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

Number of layers: 24
Number of batches: 1
Number of tokens: 28
Number of hidden units: 1024


In [16]:
# Concatenate the tensors for all layers. 
token_embeddings = torch.stack(encoded_layers, dim=0)

token_embeddings.size()

torch.Size([24, 1, 28, 1024])

In [17]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([24, 28, 1024])

In [18]:
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([28, 24, 1024])

### Creating word and sentence vectors

#### 1. Word Vectors


- Concatenate the vectors (that is, append them together) from the last four layers.
    

- Word vector(concatenate)

In [19]:
token_vecs_cat = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

Shape is: 28 x 4096


- Word vector(summing)

In [20]:
token_vecs_sum = []

for token in token_embeddings:
    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 28 x 1024


#### 2. Sentence Vector

In [21]:
token_vecs = encoded_layers[23][0]

sentence_embedding = torch.mean(token_vecs, dim=0)
print ("The sentence embedding vector of shape:", sentence_embedding.size())

The sentence embedding vector of shape: torch.Size([1024])


In [0]:
def bert_sentence_embedding(attr_text,model):

  marked_text = "[CLS] " + attr_text + " [SEP]"
  # Add the special tokens.
  marked_text = "[CLS] " + attr_text + " [SEP]"

  # Split the sentence into tokens.
  tokenized_text = tokenizer.tokenize(marked_text)

  # Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  segments_ids = [1] * len(tokenized_text)

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  
  # Predict hidden states features for each layer
  with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

  token_vecs = encoded_layers[23][0]

  sentence_embedding = torch.mean(token_vecs, dim=0)
  #print ("The sentence embedding vector of shape:", sentence_embedding.size())  

  return sentence_embedding


In [53]:
p0_str = levis_attri_desc['all_attributes'][0].replace(',',' ')

p0_str

"straight zip slim fit medium wash freewheelin suze   medium wash men's regular non stretch 100% cotton non stretch denim imported"

In [0]:
product_1_emb = bert_sentence_embedding(levis_attri_desc['all_attributes'][1].replace(',',' '),model)

In [55]:
print ("The sentence embedding vector of shape:", product_1_emb.size()) 

The sentence embedding vector of shape: torch.Size([1024])


In [56]:
product_1_emb

tensor([-0.2340,  0.2931,  0.3560,  ..., -0.5918, -0.0954, -0.0612])

In [0]:
product_2_emb = bert_sentence_embedding(levis_attri_desc['all_attributes'][7].replace(',',' '),model)

In [58]:
product_2_emb

tensor([-0.1943,  0.0131,  0.0813,  ..., -0.2270, -0.0765, -0.0145])

In [0]:
product_0_emb = bert_sentence_embedding(levis_attri_desc['all_attributes'][0].replace(',',' '),model)

In [0]:
from scipy.spatial.distance import cosine

diff_product_0_1 =  cosine(product_0_emb, product_1_emb)

diff_product_0_2 =  cosine(product_0_emb, product_2_emb)


if diff_product_0_1 > diff_product_0_2:
  print('product 0 is more similar to product 1 then product 2')
  

In [72]:
diff = product_0_emb - product_1_emb
print(cosine(product_0_emb, product_1_emb))

0.0


In [68]:
product_1_emb

tensor([-0.2340,  0.2931,  0.3560,  ..., -0.5918, -0.0954, -0.0612])

In [63]:
levis_attri_desc['all_attributes'][1]

"straight,zip,slim fit,medium wash,freewheelin suze , medium wash,men's regular,non stretch,100% cotton,non,stretch denim,imported"

In [64]:
levis_attri_desc['all_attributes'][0]

"straight,zip,slim fit,medium wash,freewheelin suze , medium wash,men's regular,non stretch,100% cotton,non,stretch denim,imported"

In [65]:
levis_attri_desc['all_attributes'][7]

'straight,550 ,zip,sits at waist,relaxed through hip and thigh,slightly tapered leg opening: 19.5",range , black,men\'s big & tall,non stretch,100% cotton,non,stretch denim,zip fly,5,pocket,wash and dry inside out with like colors, liquid detergent is recommended,imported'

In [74]:
cosine([1, 0, 0], [1,0, 0])

0.0